# Example of making animations from images of a Web Map Service

This examples requires 
* Julia 0.6 (https://julialang.org/downloads/)
* the julia package `Requests`, installable from the Julia command prompt:
```julia
Pkg.add("Requests")
```
* the tool `ffmpeg` (https://www.ffmpeg.org/download.html)

In [1]:
using Requests

# WMS base URL
wmsurl = "http://ec.oceanbrowser.net/emodnet-combined/Python/web/wms"

# WMS layer name
#layer = "Water_body_dissolved_oxygen_concentration.combined.nc*Water_body_dissolved_oxygen_concentration_L2"
layer = "Water_body_phosphate.combined.nc*Water_body_phosphate_L2"

# WMS bounding box (west,south,east,north)
bbox = "-18,30,42,66"

# short name of the title and file name of the animation
#shortname = "surface dissolved oxygen"
shortname = "surface phosphate"

# file name of the animation (without file extension)
animname = "emodnet-chemistry-$(replace(shortname," ","-"))"

# frames per second
rate = 2

# colorbar range
vmin = 0
vmax = 0.8

# year range
#years = 1980:2009;
years = 2009:2010;

# all time instances for the animation
times = ["$season $year" for season in ["winter","spring","summer","autumn"],
         year in years][:];

# loop over all time frames
for i = 1:length(times)    
    param = Dict(
        "layers" => layer,
        "request" => "GetMap",
        "width" => "800",
        "height" => "500",
        "bbox" => bbox,
        "transparent" => "false",
        "decorated" => "true",
        "crs" => "CRS:84",
        "version" => "1.3.0",
        "styles" => "cmap:jet+inverted:false+method:pcolor_flat+vmin:$(vmin)+vmax:$(vmax)+ncontours:40",
        "format" => "image/png",
        "elevation" => "0.0",
        "time" => times[i],
        "title" => "$(shortname) ($(times[i]))",
        "basemap" => "shadedrelief")

    r = Requests.get(wmsurl; query = param)

    # write into file
    open(@sprintf("%s-%05d.png",animname,i),"w") do f
        write(f, Requests.read(r))
    end
end

# generate the animation
run(`ffmpeg -y -r $(rate) -i $(animname)-%05d.png  -c:v libx264 -preset slow -crf 22 -pix_fmt yuv420p $(animname).mp4`)


INFO: Precompiling module Requests.
ffmpeg version 2.8.11-0ubuntu0.16.04.1 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.4) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enab

Embed the animation in this notebook.

In [2]:
display("text/html", string("""<video autoplay controls><source src="data:video/mp4;base64,""",
        base64encode(open(read,"$(animname).mp4")),"""" type="video/mp4"></video>"""));
